In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df = pd.read_csv('demo.csv')
df.head(3)

,tweet,clean_tweet,tweet_sentiments,clean_sentiment,sentiment_numeral
0,b'I literally watch all 3 of them as well! Eve...,b liter watch 3 well even name 6 year old ale...,positive,positive,2
1,"b""Meet Aisha: Amazon's New Sharia-Compliant Ve...",b meet aisha amazon new sharia compliant vers...,positive,positive,2
2,b'Just saw this on Amazon: Fire HD 8 Tablet wi...,b saw amazon fire hd 8 tablet alexa 8 hd disp...,positive,positive,2


In [2]:
x_train = df.loc[:28332, 'clean_tweet'].values
y_train = df.loc[:28332, 'sentiment_numeral'].values



#using two different file
x_test = df.loc[28332:, 'clean_tweet'].values
y_test = df.loc[28332:, 'sentiment_numeral'].values       



In [3]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


tokenizer_obj = Tokenizer()
total_reviews = x_train + x_test


tokenizer_obj.fit_on_texts(total_reviews)

max_length = max([len(s.split()) for s in total_reviews])

vocab_size = len(tokenizer_obj.word_index) + 1

x_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
x_test_tokens = tokenizer_obj.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_tokens, maxlen = max_length)
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_length )

G:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Flatten
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
print('Build model')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length))
model.add(GRU(units = 32, dropout=0.2, recurrent_dropout = 0.2))
#model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

Using TensorFlow backend.


Build model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
print('Train...............')
model.fit(x_train_pad, y_train, batch_size = 64,  epochs = 3, validation_data = (x_test_pad, y_test), verbose = 2)

Train...............
Instructions for updating:
Use tf.cast instead.
Train on 28333 samples, validate on 28333 samples
Epoch 1/3
 - 100s - loss: -3.1322e+00 - acc: 0.5676 - val_loss: -5.6328e+00 - val_acc: 0.4764
Epoch 2/3
 - 104s - loss: -4.6530e+00 - acc: 0.6151 - val_loss: -5.9420e+00 - val_acc: 0.5068
Epoch 3/3
 - 112s - loss: -4.8470e+00 - acc: 0.6260 - val_loss: -5.8928e+00 - val_acc: 0.5004


In [6]:
test1 = "I hate google"
test2 = "this is bad internet"

test_samples = [test1, test2]
test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen = max_length)

model.predict(x= test_samples_tokens_pad)

array([[0.00285768],
       [0.03459122]], dtype=float32)